# This notebook is aimed to analyse results of parameters evolution. 
# Therefore, parameters evolution process should be already run (either being in processed or already done).

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import copy
import json
%matplotlib inline

from deeppavlov.core.commands.utils import expand_path, parse_config
from deeppavlov.models.evolution.evolution_param_generator import ParamsEvolution

## Set here path to your config file, key main model and population size

In [ ]:
CONFIG_FILE = "../deeppavlov/configs/evolution/evolve_intents_snips.json"
KEY_MAIN_MODEL = "main"
POPULATION_SIZE = 2
    
with open(CONFIG_FILE, "r", encoding='utf8') as f:
    basic_params = json.load(f)

print("Considered basic config:\n{}".format(json.dumps(basic_params, indent=2)))

In [ ]:
evolution = ParamsEvolution(population_size=POPULATION_SIZE,
                            key_main_model=KEY_MAIN_MODEL,
                            **basic_params)
try:
    validate_best = evolution.get_value_from_config(
        evolution.basic_config, list(evolution.find_model_path(
            evolution.basic_config, "validate_best"))[0] + ["validate_best"])
    test_best = evolution.get_value_from_config(
        evolution.basic_config, list(evolution.find_model_path(
            evolution.basic_config, "test_best"))[0] + ["test_best"])
except:
    evaluation_targets = evolution.get_value_from_config(
        evolution.basic_config, list(evolution.find_model_path(
            evolution.basic_config, "evaluation_targets"))[0] + ["evaluation_targets"])
    print(evaluation_targets)
    validate_best = "valid" in evaluation_targets
    test_best = "test" in evaluation_targets

TITLE = str(evolution.models_path.stem)
print("Title name for the considered evolution is `{}`.".format(TITLE))

abs_path_to_main_models = expand_path(str(evolution.models_path).format(
        **evolution.basic_config['metadata']['variables']))
data = pd.read_csv(str(abs_path_to_main_models / "result_table.tsv"), sep='\t')
print("Number of populations: {}.".format(int(data.shape[0] / POPULATION_SIZE)))
data.fillna(0., inplace=True)

In [ ]:
MEASURES = evolution.get_value_from_config(
    evolution.basic_config, list(evolution.find_model_path(
        evolution.basic_config, "metrics"))[0] + ["metrics"])

for i in range(len(MEASURES)):
    if isinstance(MEASURES[i], dict):
        MEASURES[i] = MEASURES[i]["name"]

for measure in MEASURES:
    print("\nMeasure: {}".format(measure))
    for data_type in ["valid", "test"]:
        print("{}:".format(data_type))
        argmin = data[measure + "_" + data_type].argmin()
        argmax = data[measure + "_" + data_type].argmax()
        print("min for\t{} model on\t{} population".format(argmin % POPULATION_SIZE,
                                                             argmin // POPULATION_SIZE))
        print("max for\t{} model on\t{} population".format(argmax % POPULATION_SIZE,
                                                             argmax // POPULATION_SIZE))

## If you want to plot measures depending on population colored by evolved measure value

In [ ]:
path_to_pics = abs_path_to_main_models / "pics"
path_to_pics.mkdir(exist_ok=True, parents=True)

if validate_best:
    evolve_metric = MEASURES[0] + "_valid"
elif test_best:
    evolve_metric = MEASURES[0] + "_test"
    
cmap = plt.get_cmap('rainbow')
colors = [cmap(i) for i in np.linspace(0, 1, data.shape[0])]
color_ids = np.argsort(data.loc[:, evolve_metric].values)

ylims = [(0., 1)] * len(MEASURES)

for metric, ylim in zip(MEASURES, ylims):
    plt.figure(figsize=(12,6))
    if validate_best:
        for i in range(data.shape[0]):
            plt.scatter(i // POPULATION_SIZE, 
                        data.loc[:, metric + "_valid"].values[i], 
                        c=colors[np.where(color_ids == i)[0][0]], alpha=0.5, marker='o')
        plt.plot(np.arange(data.shape[0]//POPULATION_SIZE), 
             data.loc[:, metric + "_valid"].max() * np.ones(data.shape[0]//POPULATION_SIZE), 
             c=colors[-1])
        plt.plot(np.arange(data.shape[0]//POPULATION_SIZE), 
             data.loc[:, metric + "_valid"].min() * np.ones(data.shape[0]//POPULATION_SIZE), 
             c=colors[0])
    if test_best:
        for i in range(data.shape[0]):
            plt.scatter(i // POPULATION_SIZE, 
                        data.loc[:, metric + "_test"].values[i], 
                        c=colors[np.where(color_ids == i)[0][0]], alpha=0.5, marker='+', s=200)
        plt.plot(np.arange(data.shape[0]//POPULATION_SIZE), 
             data.loc[:, metric + "_test"].max() * np.ones(data.shape[0]//POPULATION_SIZE), "--",
             c=colors[-1])
        plt.plot(np.arange(data.shape[0]//POPULATION_SIZE), 
             data.loc[:, metric + "_test"].min() * np.ones(data.shape[0]//POPULATION_SIZE), "--",
             c=colors[0])
    

    plt.ylabel(metric, fontsize=20)
    plt.xlabel("population", fontsize=20)
    plt.title(TITLE, fontsize=20)
    plt.ylim(ylim[0], ylim[1])
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.savefig(path_to_pics.joinpath(metric + ".png"))
    plt.show()

## If you want to plot measures depending on population colored by `evolution_model_id`

####  That means model of the same `id` are of the same color.

In [ ]:
params_dictionaries = []
models_ids = []

for i in range(data.shape[0]):
    data.loc[i, "params"] = data.loc[i, "params"].replace("False", "false")
    data.loc[i, "params"] = data.loc[i, "params"].replace("True", "true")
    json_acceptable_string = data.loc[i, "params"].replace("'", "\"")
    print(json_acceptable_string)
    d = json.loads(json_acceptable_string)
    params_dictionaries.append(d)
    models_ids.append(d["evolution_model_id"])

models_ids = np.array(models_ids)
models_ids

In [ ]:
cmap = plt.get_cmap('rainbow')
colors = [cmap(i) for i in np.linspace(0, 1, len(np.unique(models_ids)))]

ylims = [(0., 1)] * len(MEASURES)

for metric, ylim in zip(MEASURES, ylims):
    plt.figure(figsize=(12,6))
    if validate_best:
        for i in range(data.shape[0]):
            plt.scatter(i // POPULATION_SIZE, 
                        data.loc[:, metric + "_valid"].values[i], 
#                         c=colors[models_ids[i]], alpha=0.5, marker='o')
                        c=colors[np.where(models_ids[i] == np.unique(models_ids))[0][0]], alpha=0.5, marker='o')
            
        plt.plot(np.arange(data.shape[0]//POPULATION_SIZE), 
             data.loc[:, metric + "_valid"].max() * np.ones(data.shape[0]//POPULATION_SIZE), 
             c=colors[-1])
        plt.plot(np.arange(data.shape[0]//POPULATION_SIZE), 
             data.loc[:, metric + "_valid"].min() * np.ones(data.shape[0]//POPULATION_SIZE), 
             c=colors[0])
    if test_best:
        for i in range(data.shape[0]):
            plt.scatter(i // POPULATION_SIZE, 
                        data.loc[:, metric + "_test"].values[i], 
                        c=colors[np.where(models_ids[i] == np.unique(models_ids))[0][0]], alpha=0.5, marker='+', s=200)
        plt.plot(np.arange(data.shape[0]//POPULATION_SIZE), 
             data.loc[:, metric + "_test"].max() * np.ones(data.shape[0]//POPULATION_SIZE), "--",
             c=colors[-1])
        plt.plot(np.arange(data.shape[0]//POPULATION_SIZE), 
             data.loc[:, metric + "_test"].min() * np.ones(data.shape[0]//POPULATION_SIZE), "--",
             c=colors[0])
    

    plt.ylabel(metric, fontsize=20)
    plt.xlabel("population", fontsize=20)
    plt.title(TITLE, fontsize=20)
    plt.ylim(ylim[0], ylim[1])
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.savefig(path_to_pics.joinpath(metric + "_colored_ids.png"))
    plt.show()


In [ ]:
cmap = plt.get_cmap('rainbow')
colors = [cmap(i) for i in np.linspace(0, 1, data.shape[0])]
color_ids = np.argsort(data.loc[:, evolve_metric].values)

for param_path in evolution.paths_to_params:
    param_name = param_path[-1]
    print(param_path, param_name)
    
    plt.figure(figsize=(12,12))
    for i in range(data.shape[0]):
        param_dict = evolution.get_value_from_config(evolution.basic_config, param_path)
        if param_dict.get("evolve_range") and param_dict.get("discrete"):
            plt.scatter(i // POPULATION_SIZE, 
                        evolution.get_value_from_config(params_dictionaries[i], param_path),
#                         + (np.random.random() - 0.5) / 2,
                        c=colors[np.where(color_ids == i)[0][0]], alpha=0.5)
        elif param_dict.get("evolve_range"):
            plt.scatter(i // POPULATION_SIZE, 
                        evolution.get_value_from_config(params_dictionaries[i], param_path),
                        c=colors[np.where(color_ids == i)[0][0]], alpha=0.5)
        elif param_dict.get("evolve_choice"):
            values = np.array(param_dict.get("values"))
            plt.scatter(i // POPULATION_SIZE, 
                        np.where(values == evolution.get_value_from_config(
                            params_dictionaries[i], param_path))[0][0],
                        c=colors[np.where(color_ids == i)[0][0]], alpha=0.5)
            plt.yticks(np.arange(len(values)), values, fontsize=20)
        elif param_dict.get("evolve_bool"):
            values = np.array([False, True])
            plt.scatter(i // POPULATION_SIZE, 
                        np.where(values == evolution.get_value_from_config(
                            params_dictionaries[i], param_path))[0][0],
                        c=colors[np.where(color_ids == i)[0][0]], alpha=0.5)
            plt.yticks(np.arange(len(values)), ["False", "True"], fontsize=20)

    plt.ylabel(param_name, fontsize=20)
    plt.xlabel("population", fontsize=20)
    plt.title(TITLE, fontsize=20)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.savefig(path_to_pics.joinpath(param_name + ".png"))
    plt.show()
    